In [1]:
!rm -rdf loader* data/* data* ../data ../data*
!wget https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/loader.py
!mkdir ../data
!wget https://www.dropbox.com/s/msqzmmaog5bstm4/dataset.npz?dl=0
!mv dataset.npz?dl=0 ../data/dataset.npz
!pip install pytorch-ignite tensorboardX

--2019-11-18 17:00:39--  https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/loader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1014 [text/plain]
Saving to: ‘loader.py’

loader.py           100%[===================>]    1014  --.-KB/s    in 0s      

2019-11-18 17:00:40 (189 MB/s) - ‘loader.py’ saved [1014/1014]

--2019-11-18 17:00:42--  https://www.dropbox.com/s/msqzmmaog5bstm4/dataset.npz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/msqzmmaog5bstm4/dataset.npz [following]
--2019-11-18 17:00:43--  https://www.dropbox.com/s/raw/msqzmmao

### Load preprocessed data

In [0]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']

n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

### Define the MF Model

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


def sample_gaussian(mu, log_var):
    var = log_var.mul(0.5).exp_()
    eps = torch.FloatTensor(var.size()).normal_()
    return mu + eps* var

def gaussian_kldiv(mu, log_var):
    kld = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
    kldloss = torch.sum(kld).mul_(-0.5)
    return kldloss


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, k=18,
                 c_bias=1.0, c_kld=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_kld = c_kld
        
        # We've added new terms here:
        self.user_mu = nn.Embedding(n_user, k)
        self.user_lv = nn.Embedding(n_user, k)
        self.item_mu = nn.Embedding(n_item, k)
        self.item_lv = nn.Embedding(n_item, k)
        
        
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        
        # *NEW: stochastically-sampled user & item vectors
        vector_user = sample_gaussian(self.user_mu(user_id), self.user_lv(user_id))
        vector_item = sample_gaussian(self.item_mu(item_id), self.item_lv(item_id))
        
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        
        # *NEW: regularization:
        user_kld = gaussian_kldiv(self.user_mu.weight, self.user_lv.weight) * self.c_kld
        item_kld = gaussian_kldiv(self.item_mu.weight, self.item_lv.weight) * self.c_kld
        
        total = loss_mse + prior_bias_user + prior_bias_item + user_kld + item_kld
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [0]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader
from datetime import datetime

#### Hyperparameters

In [6]:
lr = 1e-2
k = 10
c_bias = 1e-6
c_kld = 1e-10


log_dir = 'runs/simple_mf_08_vmf_' + str(datetime.now()).replace(' ', '_')
print(log_dir)

runs/simple_mf_08_vmf_2019-11-18_17:04:31.897801


In [7]:
writer = SummaryWriter(log_dir=log_dir)
model = MF(n_user, n_item,  k=k, c_kld=c_kld, c_bias=c_bias, writer=writer)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user_mu): Embedding(6041, 10)
  (user_lv): Embedding(6041, 10)
  (item_mu): Embedding(3953, 10)
  (item_lv): Embedding(3953, 10)
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
)

#### Run model

In [9]:
trainer.run(train_loader, max_epochs=50)

Epoch[1] Iteration[400/879] Loss: 0.62
Epoch[1] Iteration[800/879] Loss: 0.69
Epoch[1] Validation MSE: 0.80 
Epoch[2] Iteration[1200/879] Loss: 0.67
Epoch[2] Iteration[1600/879] Loss: 0.66
Epoch[2] Validation MSE: 0.80 
Epoch[3] Iteration[2000/879] Loss: 0.68
Epoch[3] Iteration[2400/879] Loss: 0.65
Epoch[3] Validation MSE: 0.80 
Epoch[4] Iteration[2800/879] Loss: 0.67
Epoch[4] Iteration[3200/879] Loss: 0.68
Epoch[4] Validation MSE: 0.80 
Epoch[5] Iteration[3600/879] Loss: 0.62
Epoch[5] Iteration[4000/879] Loss: 0.64
Epoch[5] Validation MSE: 0.80 
Epoch[6] Iteration[4400/879] Loss: 0.61
Epoch[6] Iteration[4800/879] Loss: 0.66
Epoch[6] Iteration[5200/879] Loss: 0.64
Epoch[6] Validation MSE: 0.80 
Epoch[7] Iteration[5600/879] Loss: 0.70
Epoch[7] Iteration[6000/879] Loss: 0.66
Epoch[7] Validation MSE: 0.80 
Epoch[8] Iteration[6400/879] Loss: 0.62
Epoch[8] Iteration[6800/879] Loss: 0.69
Epoch[8] Validation MSE: 0.80 
Epoch[9] Iteration[7200/879] Loss: 0.64
Epoch[9] Iteration[7600/879] Loss: